In [1]:
# Base
import itertools
from glob import glob
from tqdm import tqdm
import math
import textgrid
import random

# ML
import torch
import torch.nn.functional as F
from torch.utils.data import DistributedSampler, DataLoader
from torch.utils.tensorboard import SummaryWriter

# Local
from utils.misc import dict_to_object, plot_specgram, plot_waveform
from supervoice.audio import spectogram, load_mono_audio
from supervoice.model_duration import DurationPredictor
from supervoice.tokenizer import Tokenizer
from train_config import config

In [2]:
# Load text grid files
files = glob("datasets/vctk-aligned/**/*.TextGrid")
print(len(files))
files = files[0:10]
files = [textgrid.TextGrid.fromFile(f) for f in files]

88146


In [3]:
# Tokenizer
tokenizer = Tokenizer(config)

# Data extractor
def extract_data(src):

    # Prepare
    token_duration = 0.01
    tokens = src[1]
    time = 0
    output_tokens = []
    output_durations = []

    # Iterate over tokens
    for t in tokens:

        # Resolve durations
        ends = t.maxTime
        duration = math.floor((ends - time) / token_duration)
        time = ends

        # Resolve token
        tok = t.mark
        if tok == '':
            tok = tokenizer.silence_token

        # Apply
        output_tokens.append(tok)
        output_durations.append(duration)

    # Trim start silence
    while(output_tokens[0] == 'SIL'):
        output_durations
    if output_tokens[0] == 'SIL' and output_durations[0] > 1:
        output_durations[0] = 1
    if output_tokens[len(output_tokens) - 1] == 'SIL' and output_durations[len(output_durations) - 1] > 1:
        output_durations[len(output_durations) - 1] = 1

    # Outputs
    return output_tokens, output_durations
    
class TextGridDataset(torch.utils.data.Dataset):
    def __init__(self, files):
        self.files = files
    def __len__(self):
        return len(self.files)        
    def __getitem__(self, index):
        tg = self.files[index]

        # Load tokens/durations
        tokens, durations = extract_data(tg)
        tokens = tokenizer(tokens)
        durations = torch.Tensor(durations)

        # Calculate mask        
        mask_len = random.uniform(0.3, 0.7)
        mask_offset = random.uniform(0, 1 - mask_len)
        mask = torch.zeros(len(durations))
        mask_start = math.floor(mask_offset * len(durations))
        mask_end = math.floor((mask_offset + mask_len) * len(durations))
        mask[mask_start : mask_end] = 1
        mask = mask.bool()

        # Result
        return tokens, durations, mask


In [4]:

# Dataset, model, optimizer
device = "cpu"
dataset = TextGridDataset(files)
dataloader = DataLoader(dataset, batch_size = 1)
model = DurationPredictor(config)
model = model.to(device)
optim = torch.optim.AdamW(model.parameters(), 0.0002, betas=[0.8, 0.99])

checkpoint = torch.load(f'./output/duration_common_n_t.pt', map_location="cpu")
model.load_state_dict(checkpoint['model'])


<All keys matched successfully>

In [7]:
for i in range(10):
    # Predict
    tokens, durations, mask = dataset[i]
    predicted, loss = model(
        tokens = tokens.unsqueeze(0).to(device), 
        durations = durations.unsqueeze(0).to(device), 
        mask = mask.unsqueeze(0).to(device), 
        target = durations.unsqueeze(0).to(device)
    )
    predicted = predicted.squeeze()

    # Log
    print(f'Loss: {loss.item()}')
    print(''.join(f"{tokenizer.tokens[num]:>5}" for num in tokens.tolist()))
    print(''.join(f"{num:5}" for num in predicted.tolist()))
    print(''.join(f"{int(num):5}" for num in durations.tolist()))
    print(''.join(f"{int(num):5}" for num in mask.tolist()))

Loss: 0.06935125589370728
<SIL>    ɒ    n   d̪    ə    ɡ    ɹ   aw    n   d̪    ə    w    ɛ    s    t    b    æ    ŋ    k    w    ə    z    ə    ɡ    ɛ    n   d̪    ə    f   ow    k    ə    s    ə    v    v   aj    ə    l    ə    n    s<SIL>
    0   10    7    4    4    7    6   14    7    5    6    7    6    9    6    5    8    6    4    3    3    8    4    7    6    6    4    4   10    8    6    6    9    5    7    6   11    6    6    7    7   10    0
   85    9    7    4    4    6    5   14    7    5    5    6    8    6    6    5    6    9    7    4    4    6    4    8    7    6    2    3   10    7    8    3    8    4    6    4   12    6    4    6   10   22    3
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    1    1    1    1    1    1    1    1    1    1    1    1    1    0    0
Loss: 0.09099254757165909
<SIL>   aj    w    ə    z    ɒ    f    ə    ɫ<SIL>
    0    8    5    4    8    8   

ValueError: Tokens not found: ['spn']